In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

import warnings
warnings.simplefilter("ignore", category=DeprecationWarning)

In [2]:
propiedades = pd.read_csv('/home/mati/Desktop/set_datos_propiedades.csv')

In [3]:
propiedades = propiedades.loc[(propiedades.price_aprox_usd.notnull()) & (propiedades.superficie.notnull()),\
                             ['place_name_encoded', 'property_type_encoded','price_aprox_usd','superficie',\
                             'Year','Month','seguridad','aire','gimnasio','cochera','pileta']]

In [4]:
propiedades.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1132495 entries, 0 to 1413024
Data columns (total 11 columns):
place_name_encoded       1132495 non-null int64
property_type_encoded    1132495 non-null int64
price_aprox_usd          1132495 non-null float64
superficie               1132495 non-null float64
Year                     1132495 non-null int64
Month                    1132495 non-null int64
seguridad                1132495 non-null bool
aire                     1132495 non-null bool
gimnasio                 1132495 non-null bool
cochera                  1132495 non-null bool
pileta                   1132495 non-null bool
dtypes: bool(5), float64(2), int64(4)
memory usage: 65.9 MB


# Random Forests

In [5]:
columnas = ['superficie','place_name_encoded','property_type_encoded','seguridad','gimnasio', 'aire', 'pileta', 'cochera']
columnas_precio = columnas + ['price_aprox_usd']

In [6]:
set_entrenamiento = propiedades.loc[(propiedades.Year >= 2016) &((propiedades.Year < 2017) | (propiedades.Month < 6))\
                                    ,columnas_precio]
set_pruebas = propiedades.loc[(propiedades.Year == 2017) & (propiedades.Month == 6),columnas_precio].head(20000)

set_entrenamiento_datos = set_entrenamiento.loc[:,columnas]
set_entrenamiento_resultado = set_entrenamiento.loc[:,'price_aprox_usd']

In [7]:
randomforest = RandomForestRegressor()
randomforest.fit(set_entrenamiento_datos,set_entrenamiento_resultado)
set_pruebas.loc[:,'resultado'] = set_pruebas.loc[:,columnas].apply(lambda x: randomforest.predict(x)[0],axis = 1)
precision = randomforest.score(set_pruebas.loc[:,columnas],set_pruebas.price_aprox_usd) * 100
error = mean_squared_error(set_pruebas.price_aprox_usd,set_pruebas.resultado)
print("Precision = {:.2f} % , error = {}".format(precision, error))

Precision = 86.83 % , error = 18173048430.320446


## Ahora que tenemos una intuicion, probamos cambiando los parametros

In [8]:
columnas = ['superficie','place_name_encoded','property_type_encoded','seguridad','gimnasio', 'aire', 'pileta', 'cochera']
columnas_precio = columnas + ['price_aprox_usd']

In [9]:
set_entrenamiento = propiedades.loc[(propiedades.Year >= 2016) &((propiedades.Year < 2017) | (propiedades.Month < 6))\
                                    ,columnas_precio]
set_pruebas = propiedades.loc[(propiedades.Year == 2017) & (propiedades.Month == 6),columnas_precio].head(20000)

set_entrenamiento_datos = set_entrenamiento.loc[:,columnas]
set_entrenamiento_resultado = set_entrenamiento.loc[:,'price_aprox_usd']

res = []

In [22]:
lista_arboles = [18,19,21,22,23,24,25]
lista_feature = [0.4,0.5]

for cant_arbol in lista_arboles:
    for feature in lista_feature:
        randomforest = RandomForestRegressor(n_estimators = cant_arbol, max_features = feature)
        randomforest.fit(set_entrenamiento_datos,set_entrenamiento_resultado)
        set_pruebas.loc[:,'resultado'] = set_pruebas.loc[:,columnas].apply(lambda x: randomforest.predict(x)[0],axis = 1)
        precision = randomforest.score(set_pruebas.loc[:,columnas],set_pruebas.loc[:,'price_aprox_usd']) * 100
        error = mean_squared_error(set_pruebas.price_aprox_usd,set_pruebas.resultado)
        res.append((cant_arbol,feature,precision,error))
        print(cant_arbol,' - ',feature)

18  -  0.4
18  -  0.5
19  -  0.4
19  -  0.5
21  -  0.4
21  -  0.5
22  -  0.4
22  -  0.5
23  -  0.4
23  -  0.5
24  -  0.4
24  -  0.5
25  -  0.4
25  -  0.5


In [23]:
for r in res:
    print("Cant arboles = {}, % features = {}, precision = {:.2f} % , error = {}".format(r[0],r[1],r[2],r[3]))

Cant arboles = 5, % features = 0.7, precision = 84.56 % , error = 21303283909.34756
Cant arboles = 5, % features = 0.8, precision = 86.98 % , error = 17965657840.70977
Cant arboles = 5, % features = 1.0, precision = 85.78 % , error = 19620180384.3387
Cant arboles = 10, % features = 0.7, precision = 87.80 % , error = 16829288107.881966
Cant arboles = 10, % features = 0.8, precision = 88.25 % , error = 16211514021.839184
Cant arboles = 10, % features = 1.0, precision = 87.08 % , error = 17828865003.33896
Cant arboles = 15, % features = 0.7, precision = 88.11 % , error = 16405965237.87476
Cant arboles = 15, % features = 0.8, precision = 87.22 % , error = 17635545323.111412
Cant arboles = 15, % features = 1.0, precision = 87.26 % , error = 17575375715.987404
Cant arboles = 20, % features = 0.7, precision = 87.63 % , error = 17063372312.071825
Cant arboles = 20, % features = 0.8, precision = 87.91 % , error = 16677076812.98315
Cant arboles = 20, % features = 1.0, precision = 86.92 % , error

In [24]:
min_error = float('inf')
max_precision = 0
tupla_min_error = ()
tupla_max_precision = ()
for r in res:
    if r[3] < min_error:
        min_error = r[3]
        tupla_min_error = r
    if r[2] > max_precision:
        max_precision = r[2]
        tupla_max_precision = r
        
print("Mayor precision = Cant arboles = {}, % features = {}, precision = {:.2f} % , error = {}".\
              format(tupla_max_precision[0],tupla_max_precision[1],tupla_max_precision[2],tupla_max_precision[3]))
print("Menor error = Cant arboles = {}, % features = {}, precision = {:.2f} % , error = {}".\
              format(tupla_min_error[0],tupla_min_error[1],tupla_min_error[2],tupla_min_error[3]))

Mayor precision = Cant arboles = 20, % features = 0.4, precision = 88.63 % , error = 15692906830.660103
Menor error = Cant arboles = 20, % features = 0.4, precision = 88.63 % , error = 15692906830.660103


## Tomamos features = 0.4 y  cant = 20

In [39]:
#Probamos con varias combinaciones de columnas, y esta fue la de mejores resultados

columnas = ['superficie','place_name_encoded','property_type_encoded','seguridad','gimnasio','aire','pileta','cochera']
columnas_precio = columnas + ['price_aprox_usd']

In [40]:
set_entrenamiento = propiedades.loc[(propiedades.Year >= 2016) &((propiedades.Year < 2017) | (propiedades.Month < 6))\
                                    ,columnas_precio]
set_pruebas = propiedades.loc[(propiedades.Year == 2017) & (propiedades.Month == 6),columnas_precio].head(20000)

set_entrenamiento_datos = set_entrenamiento.loc[:,columnas]
set_entrenamiento_resultado = set_entrenamiento.loc[:,'price_aprox_usd']

In [41]:
cant_arbol = 20
feature = 0.4

In [42]:
randomforest = RandomForestRegressor(n_estimators = cant_arbol, max_features = feature)
randomforest.fit(set_entrenamiento_datos,set_entrenamiento_resultado)
set_pruebas.loc[:,'resultado'] = set_pruebas.loc[:,columnas].apply(lambda x: randomforest.predict(x)[0],axis = 1)
precision = randomforest.score(set_pruebas.loc[:,columnas],set_pruebas.price_aprox_usd) * 100
error = mean_squared_error(set_pruebas.price_aprox_usd,set_pruebas.resultado)
print("Precision = {:.2f} % , error = {}".format(precision, error))

Precision = 88.21 % , error = 16268143364.99585


# calculo con los verdaderos datos a analizar

In [43]:
analizar = pd.read_csv("/home/mati/Desktop/properati_dataset_modificado.csv")

In [44]:
analizar.loc[:,'price_usd'] = analizar.loc[:,columnas].apply(lambda x: randomforest.predict(x)[0],axis = 1)

In [45]:
resultado = analizar.loc[:,['id','price_usd']]

In [46]:
resultado.to_csv('Random_Forests_resultados', index = False)